# HW 1: Monte Carlo simulation of the Monte Hall problem

CHE 599: Intro to Data Science for Engineers  
Created by: Anthony H. Le  
Due 7 Oct 2019

### Import useful packages

In [164]:
using Random # for the 'shuffle!' function
using StatsBase # for the 'sample' function

### Data structure to represent a door with two atributes:
(i) whether or not it has been opened by the game show host to reveal a goat  
(ii) what is behind the door (a goat or a car)

In [165]:
mutable struct Door
    opened_isgoat::Bool
    behind_door::String
end

### Function that constructs and returns an array of three doors such that:
(i) two doors have a goat behind them  
(ii) one door has a car behind it  
(iii) which door has the car is chosen at uniform random  
(iv) all doors are unopened

In [282]:
function set_up_doors()
    setup = [Door(0, "goat"), Door(0, "goat"), Door(0, "car")]
    setup = shuffle!(setup)
    return setup
end

# setup = set_up_doors()

set_up_doors (generic function with 1 method)

### Function that returns an index in {1, 2, 3} at random that represents the door that the contestant chose at random

In [283]:
function pick_a_door_at_random()
    door_ids = [1, 2, 3]
    id_pick = sample(door_ids, 1, replace=false)
    return id_pick[1]
end

# random_id_pick = pick_a_door_at_random()

pick_a_door_at_random (generic function with 1 method)

### Function that opens one of the doors passed to it such that:
(i) the door that the contestant picked is not opened  
(ii) the door with the car is not opened  
(iii) if two doors are viable to open, pick one at uniform random

In [284]:
function open_a_door!(doors::Array{Door}, first_door_id_pick::Int)
    door_ids = [1, 2, 3]
    unpicked_door_ids = filter!(door_ids -> door_ids != first_door_id_pick, door_ids)
    
    unpicked_doors = [doors[i].behind_door for i = unpicked_door_ids]
    
    if any(unpicked_doors -> unpicked_doors == "car", unpicked_doors) == 1
        for i in unpicked_door_ids
            if doors[i].behind_door == "goat"
                doors[i].opened_isgoat = 1
            end
        end
    else
        host_pick = sample(doors[unpicked_door_ids], 1, replace=false)
        host_pick[1].opened_isgoat = 1
    end
    
    return doors
end

# setup = open_a_door!(setup, random_id_pick)

open_a_door! (generic function with 1 method)

### Function that returns index in {1, 2, 3} of the door that the contestant must pick if he/she switiches form his/her initial door choice to the other unopened door

In [285]:
function switch_door_pick(doors::Array{Door}, first_door_id_pick::Int64)
    door_ids = [1, 2, 3]
    unopened_doors = [doors[i].opened_isgoat for i = door_ids]
    unopened_door_ids = findall(unopened_doors -> unopened_doors == 0, unopened_doors)
    
    switch_id_pick = filter!(unopened_door_ids -> unopened_door_ids != first_door_id_pick, unopened_door_ids)
    
    return switch_id_pick[1]
end

# switch_id_pick = switch_door_pick(setup, random_id_pick)

switch_door_pick (generic function with 1 method)

### Function that returns true if the contestant picked the door with the car behind it and false otherwise

In [286]:
function contestant_wins_car(doors::Array{Door}, door_id_pick::Int)
    contest_pick = doors[door_id_pick]
    if contest_pick.behind_door == "car"
        return true
    else
        return false
    end
end

# result = contestant_wins_car(setup, switch_id_pick)

contestant_wins_car (generic function with 1 method)

### Function that uses all the functions above to simulate one Monte hall game

In [307]:
function simulate_Monte_Hall(switch::Bool; verbose::Bool=true)
    setup = set_up_doors()
    door_ids = [1, 2, 3]
    for i in door_ids
        println("Door $i has a ", setup[i].behind_door)
    end
    println(" ")
    
    random_id_pick = pick_a_door_at_random()
    println("Contestant randomly picked Door $random_id_pick")
    println(" ")
    
    open_a_door!(setup, random_id_pick)
    h = [setup[i].opened_isgoat for i = door_ids]
    host_pick = findall(h -> h == 1, h)
    host_id_pick = host_pick[1]
    println("Game show host opened Door $host_id_pick to reveal a goat")
    println(" ")
    
    switch_id_pick = switch_door_pick(setup, random_id_pick)
    
    if switch == 1
        final_id_pick = switch_id_pick
        println("Contestant switches to Door $switch_id_pick")
    else
        final_id_pick = random_id_pick
        println("Contestant sticks with initial random pick")
    end
    println(" ")
    
    if contestant_wins_car(setup, final_id_pick) == 1
        println("Contestant won")
        println(" ")
        println("##############################################")
        println(" ")
        return true
    else
        println("Contestant lost")
        println(" ")
        println("##############################################")
        println(" ")
        return false
    end
end

# result = simulate_Monte_Hall(false, verbose=false)

simulate_Monte_Hall (generic function with 1 method)

### 1000 simulations of the Monte Hall problem under the two strategies:
(i) sticking with the initial random door choice  
(ii) switching from the initial random door choice to the other unopened door

In [310]:
initial_pick_count = 0
switch_pick_count = 0

nb_sims = 1000
for s = 1:nb_sims
    result1 = simulate_Monte_Hall(false, verbose=false)
    if result1 == 1
        initial_pick_count += 1
    end
    
    result2 = simulate_Monte_Hall(true, verbose=false)
    if result2 == 1
        switch_pick_count +=1
    end
end

initial_pick_ratio = initial_pick_count / nb_sims
switch_pick_ratio = switch_pick_count / nb_sims

println("Contestant won $initial_pick_count times ($initial_pick_ratio) when sticking with initial random pick")
println(" ")
println("Contestant won $switch_pick_count times ($switch_pick_ratio) when switching picks")

Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 3
 
Game show host opened Door 1 to reveal a goat
 
Contestant switches to Door 2
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 3
 
Game show host opened Door 2 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 2
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 

 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant switches to Door 3
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 3
 
Game show host opened Door 1 to reveal a goat
 
Contestant switches to Door 2
 
Contestant lost
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
####

Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 1
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 2
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 3
 
Game show host opened Door

Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 1
 
Contestant lost
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant switches to Door 3
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Cont

 
Game show host opened Door 1 to reveal a goat
 
Contestant switches to Door 2
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 2
 
Game show host opened Door 1 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant switches to Door 3
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 1
 
Game show host opened

...

Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 3
 
Game show host opened Door 2 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 3
 
Game show host opened Door 2 to reveal a goat
 
Contestant switches to Door 1
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant switches to Door 3
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 

##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 2
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 1
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 3
 
Game show host opened Door 2 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
########################

 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 2
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 2
 
Game show host opened Door 1 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant switches to Door 3
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 1
 
Game show host opened Door 2 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
####

 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 1
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 3
 
Game show host opened Door 1 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 2
 
Game show host opened Door 1 to reveal a goat
 
Contestant switches to Door 3
 
Contestant won
 
##############################################
 
Door 1 has a car
Door 2 has a goat
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened Door 3 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant won
 
#######################

 
Game show host opened Door 3 to reveal a goat
 
Contestant switches to Door 1
 
Contestant won
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 3
 
Game show host opened Door 1 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 2
 
Game show host opened Door 1 to reveal a goat
 
Contestant switches to Door 3
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a goat
Door 3 has a car
 
Contestant randomly picked Door 2
 
Game show host opened Door 1 to reveal a goat
 
Contestant sticks with initial random pick
 
Contestant lost
 
##############################################
 
Door 1 has a goat
Door 2 has a car
Door 3 has a goat
 
Contestant randomly picked Door 1
 
Game show host opened